In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import string
import numpy as np
import gzip
import seaborn as sns
np.random.seed(99)
RANDOM_STATE = 99
import datetime
pd.set_option('display.max_rows', 151)


# Import vectorizing and modeling tools in preparation for modeling steps
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.feature_extraction import text, stop_words
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
import time

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import spacy
import en_core_web_lg
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import roc_curve, auc

# Import CountVectorizer and TFIDFVectorizer from feature_extraction.text.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [47]:
df = pd.read_csv('../data/locsubs.csv')

In [48]:
df.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
author,thesimranvenkat,Persephonal,curlzruleboysdrool,B1gg-E,Aphander
author_flair_css_class,NaN,NaN,NaN,NaN,NaN
author_flair_richtext,[],[],[],[],[]
author_flair_text,NaN,NaN,NaN,NaN,NaN
author_flair_type,text,text,text,text,text
can_mod_post,False,False,False,False,False
contest_mode,False,False,False,False,False
created_utc,1530883295,1532457644,1533044929,1534213933,1535246552
domain,i.redd.it,self.curlyhair,self.curlyhair,self.curlyhair,self.curlyhair


In [49]:
df.dtypes

Unnamed: 0                         int64
author                            object
author_flair_css_class            object
author_flair_richtext             object
author_flair_text                 object
author_flair_type                 object
can_mod_post                        bool
contest_mode                        bool
created_utc                        int64
domain                            object
full_link                         object
gilded                           float64
id                                object
is_crosspostable                    bool
is_original_content                 bool
is_reddit_media_domain              bool
is_self                             bool
is_video                            bool
link_flair_background_color       object
link_flair_css_class              object
link_flair_richtext               object
link_flair_template_id            object
link_flair_text                   object
link_flair_text_color             object
link_flair_type 

In [50]:
type(df['selftext'])

pandas.core.series.Series

In [51]:
nlp = spacy.load('en_core_web_sm', disable = 'parser')

In [56]:
def tokenize(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    return ' '.join(tokens)



In [57]:
df['tokens'] = df['selftext'].apply(tokenize)

TypeError: object of type 'float' has no len()

TypeError: object of type 'bool' has no len()

In [34]:
df['tokens']

Series([], Name: tokens, dtype: object)

In [58]:
df['selftext'].isnull().sum()

14

In [59]:
df = df.dropna(axis=0, how = 'any')

In [60]:
df.head()

,Unnamed: 0,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,can_mod_post,contest_mode,created_utc,domain,...,all_awardings,total_awards_received,allow_live_comments,steward_reports,awarders,og_description,og_title,author_premium,treatment_tags,upvote_ratio
